<a href="https://colab.research.google.com/github/luuleitner/dasIT/blob/main/beamform_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1>Plane Wave Beamforming Tutorial</h1>

This is a hands-on introduction to plane wave ultrasound beamforming. During this exercise we will look at the raw ultrasound data acquired and how to convert it into actual images. 

<br>

<h2>Main Take Aways</h2>

- understand ultrasonic raw data,
- know the necessary processing elements of a plane wave beamformer, and
- be able to perform a quality estimation of an ultrasound image.

<br>

<h2>Supplemental Information</h2>

This practical example is part of the ***Graz University of Technology*** lecture series ***Development of Electronic Systems*** and the ***Fundamentals of Biomedical Engineering Laboratory***. 

This program is free software and licensed under the ***Apache License v2.0*** (see the github LICENSE file for details). It is also citable: ***Leitner, C. (2022). New directions in recording and processing electro-mechanical signals from the human body. Dissertation. Graz University of Technology.***

<br>

<h2>Uncomplete List of Other Free Ultrasound Ressources:</h2>

*   <a href="http://www.k-wave.org/">k-wave ultrasound simulator</a> - free MATLAB and C++ implementations
*   <a href="https://field-ii.dk/">field II ultrasound simulator</a> - free MATLAB implementation
*   <a href="https://github.com/Sergio5714/pybf">pybf - Python beamformer</a> - optimized for short processing times
*   <a href="https://www.biomecardio.com/MUST/">MATLAB ultrasound toolbox</a> - free MATLAB beamformer<br><br>

<br><br>

I wish everyone a great dive into the topic, and please do not hesitate to <a href="mailto:christoph.leitner@tugraz.at">contact</a> me in case of **any** questions!

yours,<br>
Christoph<br><br>

<br>
<br>


---




# Introduction


<br>

**But what is beamforming anyway?** Essentially, this involves generating ultrasonic waves (beam patterns) for a transmit event (TX) and reconstructing images from the reflected ultrasonic waves or received signals (RX) on the piezo. 

<br>

**Simplifications:** For ultrasound image reconstruction, we make some additional assumptions, e.g..
- sound propagates throughout the tissue at a constant speed of sound,
- sound propagates in straight lines,
- sound is attenuated by the same amount throughout the tissue.

However, all of these assumptions are only model simplifications. Therefore, we must be aware that image distortion can occur if one of these basic assumptions are not met.

<br>

<h2>Plane wave beamforming principles</h2>

This method of beamforming is probably the easiest way to get started. In **plane wave beamforming**, all transducer elements are activated simultaneously, creating a single, unfocused ultrasonic plane wave that propagates in a straight line into the tissue (see figure below). 

<br>

<figure>
<center>
<img src='https://raw.githubusercontent.com/luuleitner/dasIT/main/example_data/ColabBFfigures/PlaneWaveBF.gif'/ width="1200">
</figure>

<br>

**How does this simplify things?** With a beamformer we have to solve the inverse problem, i.e. the transfer of the ultrasonic signal amplitudes from the time-channel space into an image space. In the case of beamforming plane waves, the solution simplifies to straightforward geometric formulations (we will have a look at them later on).

<br>

---

# 1.Getting Started


```
[#] Areas shown like this are executable code. Use the mousover play button to run these cells.
```

## Installation
First we need to install the beamformer package from the the <a href="https://github.com/luuleitner/dasIT">GitHub repository</a>:

In [ ]:
# Fetch the newest dasIT package from the github repository

!pip install git+https://github.com/luuleitner/dasIT

Next we load all other necessary libraries into our Colab notebook:

In [ ]:
# import necessary packages
import os
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt

# to run dasIT, we need to import the necessary commands from the library
from dasIT.data.loader import RFDataloader, TDloader, TGCloader
from dasIT.features.transducer import transducer
from dasIT.features.medium import medium
from dasIT.features.tgc import tg_compensation
from dasIT.src.delays import planewave_delays
from dasIT.src.apodization import apodization
from dasIT.src.das_bf import RXbeamformer
from dasIT.features.signal import RFfilter, fftsignal, analytic_signal
from dasIT.features.image import interp_lateral
from dasIT.visualization.signal_callback import amp_freq_1channel, amp_1channel
from dasIT.visualization.image_callback import plot_signal_grid, plot_signal_image
from dasIT.features.signal import logcompression

## Download the example dataset

In [ ]:
# Download the dataset from github

# The data was obtained in the course of the work for:
# Leitner et al. 2020, "Detection of Motor Endplates in Deep and Pennate Skeletal Muscles in-vivo using Ultrafast Ultrasound",
# 2020 IEEE International Ultrasonics Symposium (IUS).
#

rfdata_path = '/content/rfdata'

if os.path.exists(rfdata_path) == False:
  os.mkdir(rfdata_path)
  os.chdir(rfdata_path)
  !wget -i https://raw.githubusercontent.com/luuleitner/dasIT/main/example_data/CIRSphantom_GE9LD_VVantage/COLABdownload_url.txt

os.chdir(rfdata_path)

### Compilation of the data set
Five frames were captured on a **Verasonics Vantage 256** ultrasound research system, using a **GE-9LD** transducer, and a **CIRS General Purpose** ultrasound phantom.

The figure below shows the reconstructed ultrasonic image (left) captured on an ultrasound phantom (center) with a GE-9LD transducer (right).

<figure>
<center>
<img src='https://raw.githubusercontent.com/luuleitner/dasIT/main/example_data/ColabBFfigures/ExperimentalSetup.jpg'/ width="1200">
</figure>

# 2.Define Hardware and Imaging Medium

To get started, we need to provide our software several static parameters. In particular, we have to define the used ultrasound hardware (e.g. transducer design,..) the emitted ultrasound signal (e.g. used ultrasound frequency,..) and the definition of the medium (e.g. speed of sound, size, pixels,..) below the ultrasound lens.

We assume a constant speed of sound (**v = 1540 m/s**) in the tissue. This model assumption allows us to infer the tissue depth of the reflected echo by the relationship v=s/t and the measurement of the delay time of the reflected sound (Puls-Echo method).

## Transducer

Based on the transducer specs given above fill in the necessary parameters below:

In [ ]:
# dasIT transducer
physical_transducer = TDloader('transducer.csv')
dasIT_transducer = transducer(center_frequency_hz = 5000000,  # <--- FILL IN CENTER FREQUENCY OF THE TRANSDUCER IN [Hz]
                              bandwidth_hz=physical_transducer.transducer['bandwidth'].dropna().to_numpy(dtype='float', copy=False),    # [Hz]
                              adc_ratio=4,  # [-]
                              transducer_elements_nr = 192, # <--- FILL IN THE NUMBER OF TRANSDUCER ELEMENTS [#]
                              element_pitch_m = 0.00023, # <--- FILL IN THE ELEMENT PITCH IN [m]
                              pinmap=physical_transducer.transducer['pinmap'].dropna().to_numpy(dtype='int', copy=False),   # [-]
                              pinmapbase=1, # [-]
                              elevation_focus=0.028, # [m]
                              focus_number=None,
                              totalnr_planewaves=1,     # [-]
                              planewave_angle_interval=[0,0],   # [rad]
                              axial_cutoff_wavelength=5,  # [#]
                              speed_of_sound_ms = 1540)  # <--- FILL IN THE SPEED OF SOUND IN [m/s]

In [ ]:
# Print the transducer specifications

print(f'Transducer properties:')
print()
vars(dasIT_transducer)

## Medium

The size of the medium we are imaging is usually specified in "wavelength numbers" via the relationship lambda = c/f. We don't worry too much about these definitions and simply load the specifications of the imaging medium.

In [ ]:
# dasIT medium
dasIT_medium = medium(speed_of_sound_ms = 1540, # [m/s]
                      center_frequency = dasIT_transducer.center_frequency, # [Hz]
                      sampling_frequency = dasIT_transducer.sampling_frequency, # [Hz]
                      max_depth_wavelength = 177,   # [#]
                      lateral_transducer_element_spacing = dasIT_transducer.lateral_transducer_spacing, # [m]
                      axial_extrapolation_coef = 1.05,  # [-]
                      attenuation_coefficient= 0.75,   # [dB/(MHz^y cm)]
                      attenuation_power=1.5   # [-]
                      )

# 3.Data Wrangling

From an ultrasound research system, we usually get more information than we actually need (due to the memory allocation of the system), and the channel data is usually unsorted. Therefore, we need to "clean up" the before we begin. So we cut off unassigned samples (zero values) and sort our channel data in ascending order from 1-192. 

<figure>
<center>
<img src='https://raw.githubusercontent.com/luuleitner/dasIT/main/example_data/ColabBFfigures/DataPreperation.jpg'/ width="1200">
</figure>


In [ ]:
### Load RF Data
RFdata = RFDataloader('CIRS_phantom.h5')

### Preprocess (Clip and Sort) RF Data
# Samples start: at first recorded echo (number of wavelength distance is provided from vendor)
# -> null out the rest to not overshadow the real results
# Samples end: at penetration depth -> clip rest of samples without data
# If necessary sort the transducer channels according to the pin map to get the channels first-last channel
RFdata.signal[:dasIT_transducer.start_depth_rec_samples, :, :] = 0
RFdata.signal = RFdata.signal[:dasIT_medium.rx_echo_totalnr_samples, dasIT_transducer.transducer_pinmap, :]

print(f'Channels of transducer: {RFdata.signal.shape[1]}')
print(f'Samples per channel: {RFdata.signal.shape[0]}')
print(f'Number of frames: {RFdata.signal.shape[2]}')

## Exploring raw data - Beamforming Principle

If we now plot the sorted and trimmed channel data over time (left image), point-like echoes (phantom reflectors in the right image) show up, hyperbolic in the x,t plots.

**With our beamformer, we now need to solve the inverse problem and transfer the raw ultrasound data (time and channel space) into an image (width and height space)**.

<figure>
<center>
<img src='https://raw.githubusercontent.com/luuleitner/dasIT/main/example_data/ColabBFfigures/Beamformer_xt_xz.jpg'/ width="1200">
</figure>

### Exploring data of a single channel
Let's have a look at the raw data of **Channel 156**. We plot the signal amplitude over time.

In [ ]:
cutoff_lens = 90
cutoff_depth = len(RFdata.signal) - 700

channel = 156
dbrange = 55
signal = logcompression(RFdata.signal[:,:,0], dbrange)

fig = plt.figure(figsize=(10, 4), dpi=100)
ax_1 = fig.add_subplot(121)
ax_1.imshow(signal[90:-700, :],
            aspect=1,
            cmap='gray')

ax_1.set_xlabel('Transducer Element [#]', fontsize=15, fontweight='bold', labelpad=10)
ax_1.set_ylabel('Passing Time (Sample [#])', fontsize=15, fontweight='bold', labelpad=10)
ax_1.xaxis.tick_top()
ax_1.xaxis.set_label_position('top')
ax_1.axvline(x=channel, color='red')
ax_1.axhline(y=0, color='blue', lw=5)
ax_1.axhline(y=cutoff_depth-cutoff_lens-2, color='blue')


ax_2 = fig.add_subplot(122)
ax_2.plot(RFdata.signal[:,channel,0], 'r')
ax_2.set_xlabel('Samples [#]')
ax_2.set_ylabel('Signal [V]')
ax_2.set_title(f'RF-data channel {channel}')
ax_2.axvline(x=cutoff_lens, color='blue',)
ax_2.axvline(x=cutoff_depth, color='blue')

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()
plt.show()

print('')
print('Channel 156 is indicated in red color.\nThe visible area in the time-signal map on the left is marked with blue lines in the channel plot on the right side.')

# 4.Signal Processing

### Time Gain Compensation

We assume that acoustic waves are absorbed to the same extent everywhere in the tissue (which is not formally correct, since different tissue types absorb to different amounts). The pressure amplitudes of acoustic waves lose energy as they propagate through tissue due to attenuation. Therefore, reflections from deeper structures appear weaker. For this reason, time gain correction is usually applied to the recorded RF data by adding a time-dependent gain to compensate for the attenuation losses.

<figure>
<center>
<img src='https://raw.githubusercontent.com/luuleitner/dasIT/main/example_data/ColabBFfigures/TGC.jpg'/ width="1200">
</figure>

In [ ]:
# Load and apply tgc-waveform
tgc_cntrl_points = TGCloader(controlpt_path='tgc_cntrl_pt.csv')
TGCsignals = tg_compensation(signals=RFdata.signal,
                             medium=dasIT_medium,
                             center_frequency=dasIT_transducer.center_frequency,
                             cntrl_points=tgc_cntrl_points,
                             mode='points')

In [ ]:
# Plot channel 156
fig = plt.figure(figsize=(5, 3), dpi=100)
ax_1 = fig.add_subplot(211)
ax_1.plot(RFdata.signal[:,channel,0])
ax_1.set_xlabel('Samples [#]')
ax_1.set_ylabel('Signal [V]')
ax_1.set_title(f'RF-data channel {channel}')

ax_2 = fig.add_subplot(212)
ax_2.plot(TGCsignals.signals[:,channel,0],'r')
ax_2.set_xlabel('Samples [#]')
ax_2.set_ylabel('Signal [V]')
ax_2.set_title(f'TGC compensated RF-data channel {channel}')

plt.tight_layout()
plt.show()

###Filtering

To eliminate unwanted noise, we filter our raw data with a FIR bandpass filter and Gaussian window. We use the upper and lower limits of the transducer bandwidth as cutoff frequencies.

In [ ]:
### Filter RF Data
RFdata_filtered = RFfilter(signals=TGCsignals.signals,
                           fcutoff_band=dasIT_transducer.bandwidth,
                           fsampling=dasIT_transducer.sampling_frequency,
                           type='gaussian',
                           order=10)

In [ ]:
# Plot channel 156
fftFil = fftsignal(RFdata_filtered.signal[:,channel,0], dasIT_transducer.sampling_frequency)

fig = plt.figure(figsize=(6, 5), dpi=100)
ax_1 = fig.add_subplot(311)
ax_1.plot(RFdata.signal[:,channel,0])
ax_1.set_xlabel('Samples [#]')
ax_1.set_ylabel('Signal [V]')
ax_1.set_title(f'RF-data channel {channel}')

ax_2 = fig.add_subplot(312)
ax_2.plot(RFdata_filtered.signal[:,channel,0],'r')
ax_2.set_xlabel('Samples [#]')
ax_2.set_ylabel('Signal [V]')
ax_2.set_title(f'Filtered RF-data channel {channel}')
 
ax_3 = fig.add_subplot(313)
ax_3.plot(fftFil[0], fftFil[1], 'r')
ax_3.set_xlabel('Frequency [MHz]')
ax_3.set_ylabel('Power [W/Hz]')
ax_3.set_title(f'FFT channel {channel}')

plt.tight_layout()
plt.show()

### Convert to analytical signal

In the ultrasound domain, we prefer to use signal envelopes to represent data We display the instantaneous energy distributions from fluctuating raw signals. Therefore, we must first calculate the analytical signal of the raw data using the Hilbert transformation.

In [ ]:
####################################################################
#------------------------ Analytical Signal -----------------------#

### Hilbert Transform
RFdata_analytic = analytic_signal(np.squeeze(RFdata_filtered.signal), interp=False)

In [ ]:
# Plot channel 156
fftFil = fftsignal(RFdata_filtered.signal[:,channel,0], dasIT_transducer.sampling_frequency)

fig = plt.figure(figsize=(5, 3), dpi=100)
ax_1 = fig.add_subplot(211)
ax_1.plot(RFdata.signal[:,channel,0])
ax_1.set_xlabel('Samples [#]')
ax_1.set_ylabel('Signal [V]')
ax_1.set_title(f'RF-data channel {channel}')

ax_2 = fig.add_subplot(212)
ax_2.plot(RFdata_analytic[:,channel,0].real,'r')
ax_2.plot(abs(RFdata_analytic[:,channel,0]),'g', label='envelope')
ax_2.set_xlabel('Samples [#]')
ax_2.set_ylabel('Signal [V]')
ax_2.set_title(f'Analytic signal channel {channel}')
ax_2.legend(loc='lower right')

plt.tight_layout()
plt.show()

# 5.Beamforming


To solve the beamforming operation, signal amplitudes must be transferred from the (x,time) domain into the (z,x) plane. The figure below depicts the necessary mathematical operations to calculate the time delays needed to determine
the signal amplitudes at each pixel point:

<figure>
<center>
<img src='https://raw.githubusercontent.com/luuleitner/dasIT/main/example_data/ColabBFfigures/PWbeamforming.jpg'/ width="800">
</figure>


### Acount for element directivity and focus with an apodization table


In [ ]:
####################################################################
#-------------------------- Apodization Table --------------------------#

apodization = apodization(delays=None,
                          medium=dasIT_medium.medium,
                          transducer=dasIT_transducer,
                          apo='rec',
                          angles=dasIT_transducer.planewave_angles())

### Setup delay table for signal transformation

In [ ]:
####################################################################
#-------------------------- Delay Tables --------------------------#

### DAS delay tabels for tilted planewaves
delay_table = planewave_delays(medium=dasIT_medium.medium,
                               sos=dasIT_medium.speed_of_sound,
                               fsampling=dasIT_transducer.sampling_frequency,
                               angles=dasIT_transducer.planewave_angles())

###Beamforming

In [ ]:
####################################################################
#-------------------------- Beamforming ---------------------------#
start_das_timing = datetime.now()

# Mask images areas in axial direction which have been included for reconstruction
# but are not part of the actual image.
RFsignals = RFdata_analytic[:,:,0]

RFsignals = np.expand_dims(RFsignals, 2)
RFsignals = np.repeat(RFsignals, RFsignals.shape[1], axis=2)
RFsignals = np.expand_dims(RFsignals, 3)

BFsignals = RXbeamformer(signals=RFsignals,
                         delays=delay_table.sample_delays,
                         apodization=apodization.table)

# 6.Image Formation

In [ ]:
####################################################################
#------------------------ Image Formation --------------------------

# Envelope
BFsignals.envelope = abs(BFsignals.frame)

# Interpolate over Lateral space
BFsignals.interpolated = interp_lateral(signals=BFsignals.envelope,
                                        transducer=dasIT_transducer,
                                        medium=dasIT_medium,
                                        scale=3)


# Plot image
plot_signal_grid(signals=BFsignals.interpolated.signals_lateral_interp,
                 axis_vectors_xz=BFsignals.interpolated.imagegrid_mm,
                 axial_clip=[dasIT_transducer.start_depth_rec_m, None],
                 compression=True,
                 dbrange=58,
                 path=rfdata_path)

# 7.Calculate Image Resolution

For the indicated point target in the near field (figure below) calculate the axial and lateral resolutions of your beamformed image. 

<figure>
<center>
<img src='https://raw.githubusercontent.com/luuleitner/dasIT/main/example_data/ColabBFfigures/Resolution.jpg'/ width="300">
</figure>


Instructions: 
- Use a ruler and measure the full width (start and end points) of your reflection target in the lateral and axial directions. 
- One full width is equal to approximately 6 standard deviations (sigma).
- Divide the full width by 6 to get the approximate value of one sigma.
- Calculate the full width at half maximum (FWHM) using a Gaussian distribution: FWHM = 2*sqrt(2 *ln2) * sigma
